# Cleaning Intel Data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Stock data we downloaded from Marketwatch and then put together in a Google Sheet which we then downloaded for the past 15 years
intel_stock = pd.read_csv("IntelStockData.csv")
intel_stock.head()

,Date,Open,High,Low,Close,Volume
0,1/2/2026,37.77,39.86,37.76,39.38,"95,396,445"
1,12/31/2025,37.40,37.58,36.87,36.90,"51,524,680"
2,12/30/2025,36.91,38.26,36.82,37.30,"61,935,281"
3,12/29/2025,36.01,36.80,35.82,36.68,"38,062,301"
4,12/26/2025,36.17,36.49,35.85,36.20,"28,779,400"


In [118]:
# Analyst data that we retrieved from the Google BigQuery; I am limiting to Intel because that is the company I am working on
analyst_data = pd.read_csv("AllAnalystData.csv", dtype={'price_target_before': str, 'price_target_after':str})
analyst_data = analyst_data.iloc[:,0:9]
intel_analyst_data = analyst_data[analyst_data.iloc[:,2] == "INTC"]
intel_analyst_data.head()

,date,company_Name,ticker,broker,analytst,rating_before,rating_after,price_target_before,price_target_after
835,7/6/2020,Intel Corp,INTC,NaN,JAMES COVELLO,Neutral,Sell,65,54
836,4/15/2021,Intel Corp,INTC,NaN,CHRIS CASO,Mkt Perform,Underperf,NaN,NaN
837,1/25/2019,Intel Corp,INTC,WELLS FARGO,AARON RAKERS,NaN,OUTPERFORM,NaN,55
838,5/9/2019,Intel Corp,INTC,WELLS FARGO,AARON RAKERS,,NEGATIVE,60,55
839,4/24/2020,Intel Corp,INTC,WELLS FARGO,AARON RAKERS,,EQUAL WEIGHT,56,60


In [120]:
# Cleaning instances where a special arrow character is used so that I can fully clean all NA's later
intel_analyst_data.iloc[:,7] = intel_analyst_data.iloc[:,7].replace('70 » 60         ', 70)
intel_analyst_data.iloc[:,8] = intel_analyst_data.iloc[:,8].replace('70 » 60', 60)

intel_analyst_data.iloc[:,7] = intel_analyst_data.iloc[:,7].replace('50 » 53         ', 50)
intel_analyst_data.iloc[:,8] = intel_analyst_data.iloc[:,8].replace('50 » 53', 53)

intel_analyst_data.iloc[:,7] = intel_analyst_data.iloc[:,7].replace('50 » 45         ', 50)
intel_analyst_data.iloc[:,8] = intel_analyst_data.iloc[:,8].replace('50 » 45', 45)

intel_analyst_data.iloc[:,7] = intel_analyst_data.iloc[:,7].replace('47 » 48         ', 47)
intel_analyst_data.iloc[:,8] = intel_analyst_data.iloc[:,8].replace('47 » 48', 48)

intel_analyst_data.iloc[:,7] = intel_analyst_data.iloc[:,7].replace('58 » 60         ', 58)
intel_analyst_data.iloc[:,8] = intel_analyst_data.iloc[:,8].replace('58 » 60', 60)

In [125]:
# Command to replace the empty cells with NaN values so I can now filter DataFrame 
intel_analyst_data.iloc[:,7] = intel_analyst_data.iloc[:,7].str.strip().replace('',np.nan)
intel_analyst_data.iloc[:,8] = intel_analyst_data.iloc[:,8].str.strip().replace('',np.nan)

In [134]:
# Taking out all rows that have an NaN in price_target_after
intel_nas = intel_analyst_data.isna()
clean_intel = intel_analyst_data[intel_nas.iloc[:,8] == False]
clean_intel.head()

,date,company_Name,ticker,broker,analytst,rating_before,rating_after,price_target_before,price_target_after
835,7/6/2020,Intel Corp,INTC,NaN,JAMES COVELLO,Neutral,Sell,65,54
837,1/25/2019,Intel Corp,INTC,WELLS FARGO,AARON RAKERS,NaN,OUTPERFORM,NaN,55
838,5/9/2019,Intel Corp,INTC,WELLS FARGO,AARON RAKERS,,NEGATIVE,60,55
839,4/24/2020,Intel Corp,INTC,WELLS FARGO,AARON RAKERS,,EQUAL WEIGHT,56,60
840,7/24/2020,Intel Corp,INTC,WELLS FARGO,AARON RAKERS,,EQUAL WEIGHT,60,55


In [137]:
# This renamed the analyst column in the Analyst data correctly and also made the date column in datetime values
clean_analyst_data = clean_intel.rename(columns={'analytst':'analyst'})
clean_analyst_data['date'] = pd.to_datetime(clean_analyst_data['date'])

In [147]:
# This renamed the date column in the Stock data correctly, made the date column in datetime values and created a variable for the stock data columns necessary
intel_stock = intel_stock.rename(columns={'Date':'date'})
intel_stock['date'] = pd.to_datetime(intel_stock['date'])
intel_stock_data_needed = intel_stock[['date', 'Close', 'High', 'Low']]

In [148]:
analyst_data_date_sort = clean_analyst_data.sort_values(by='date', ascending=False)
analyst_data_date_sort.head()

,date,company_Name,ticker,broker,analyst,rating_before,rating_after,price_target_before,price_target_after
1467,2025-07-14,Intel Corp,INTC,NaN,TIMOTHY ARCURI,Neutral,Neutral,25,25
985,2025-07-07,Intel Corp,INTC,NaN,CHRISTOPHER DANELY,Neutral,Neutral,24,24
1339,2025-05-21,Intel Corp,INTC,NaN,ROSS SEYMORE,Hold,Hold,23,23
1211,2025-04-30,Intel Corp,INTC,NaN,KEVIN CASSIDY,Sell,Sell,14,14
852,2025-04-28,Intel Corp,INTC,WELLS FARGO,AARON RAKERS,NaN,EQUAL WEIGHT,NaN,22


In [149]:
intel_data = pd.merge(left=analyst_data_date_sort, right=intel_stock_data_needed, on='date', how='inner')

In [150]:
intel_data

,date,company_Name,ticker,broker,analyst,rating_before,rating_after,price_target_before,price_target_after,Close,High,Low
0,2025-07-14,Intel Corp,INTC,NaN,TIMOTHY ARCURI,Neutral,Neutral,25,25,23.30,23.40,22.88
1,2025-07-07,Intel Corp,INTC,NaN,CHRISTOPHER DANELY,Neutral,Neutral,24,24,22.00,22.39,21.86
2,2025-05-21,Intel Corp,INTC,NaN,ROSS SEYMORE,Hold,Hold,23,23,20.69,21.63,20.59
3,2025-04-30,Intel Corp,INTC,NaN,KEVIN CASSIDY,Sell,Sell,14,14,20.10,20.18,19.55
4,2025-04-28,Intel Corp,INTC,WELLS FARGO,AARON RAKERS,NaN,EQUAL WEIGHT,NaN,22,20.51,21.08,20.05
...,...,...,...,...,...,...,...,...,...,...,...,...
690,2013-01-18,Intel Corp,INTC,CRT CAPITAL,VIJAY RAKESH,NaN,NEUTRAL,NaN,18,21.25,21.44,21.03
691,2012-10-17,Intel Corp,INTC,NaN,MARK LIPACIS,Hold,Hold,24,24,21.79,21.98,21.27
692,2012-09-10,Intel Corp,INTC,NaN,CHRISTOPHER DANELY,Neutral,Neutral,22,22,23.26,24.60,23.17
693,2012-07-18,Intel Corp,INTC,NaN,MARK LIPACIS,Hold,Hold,29,29,26.21,26.42,25.05
